In [4]:
import keras
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import numpy as np
import pandas as pd
import glob

df = pd.read_excel('fer2013.xlsx')

In [5]:
from math import ceil 

indices = list(range(len(df[df['Usage'] == 'Training'])))
np.random.shuffle(indices)
train_indices = indices[ceil(len(indices)/5):]
valid_indices = indices[:ceil(len(indices)/5)]
labels = df.emotion

In [6]:
one_hot_labels = keras.utils.to_categorical(df.emotion, num_classes=len(df.emotion.unique()))
test_labels = one_hot_labels[~df.index.isin(indices)]
train_labels = one_hot_labels[train_indices]
valid_labels = one_hot_labels[valid_indices]

In [7]:
def to_rgb1a(im):
    # Convert to RGB
    w, h = im.shape
    ret = np.empty((w, h, 3), dtype=np.float16)
    ret[:, :, 2] =  ret[:, :, 1] =  ret[:, :, 0] =  im
    return ret

In [8]:
%%time
df['pixels'] = df['pixels'].apply(lambda x: to_rgb1a(np.reshape(np.array(x.split()),(48,48))))
df.head()

CPU times: user 1min 23s, sys: 368 ms, total: 1min 23s
Wall time: 1min 23s


In [9]:
def paths_to_tensor(arrs):
    list_of_tensors = [np.expand_dims(arr, axis = 0) for arr in arrs]
    return np.vstack(list_of_tensors)

In [10]:
img_input = preprocess_input(paths_to_tensor(df['pixels']))

In [44]:
import keras
from keras.applications.vgg19 import VGG19
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2,padding='same', activation='relu', input_shape=img_input[train_indices].shape[1:]))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dense(512, activation='relu'))
model.add(GlobalAveragePooling2D())
model.add(Dense(7, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 48, 48, 16)        208       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 24, 24, 32)        2080      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
dense_13 (Dense)             (None, 6, 6, 512)         33280     
__________

In [49]:
from keras.callbacks import ModelCheckpoint

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='C:/Users/Alvin/.keras/models/weights.best.scratch2.face-{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(img_input[train_indices], train_labels,
          epochs=10,
          batch_size=20,
          validation_data=(img_input[valid_indices], valid_labels), callbacks = [checkpointer])
model.save_weights('bottleneck_fc_model.h5')

Train on 22967 samples, validate on 5742 samples
Epoch 1/10
22967/22967 [==============================] - 48s - loss: 1.3999 - acc: 0.4702 - val_loss: 1.7292 - val_acc: 0.3553
Epoch 2/10
22967/22967 [==============================] - 48s - loss: 1.3781 - acc: 0.4798 - val_loss: 1.5287 - val_acc: 0.4626
Epoch 3/10
22967/22967 [==============================] - 48s - loss: 1.3831 - acc: 0.4741 - val_loss: 1.4884 - val_acc: 0.4490
Epoch 4/10
22967/22967 [==============================] - 49s - loss: 1.3817 - acc: 0.4818 - val_loss: 1.6534 - val_acc: 0.4047
Epoch 5/10
22967/22967 [==============================] - 54s - loss: 1.3868 - acc: 0.4803 - val_loss: 1.5193 - val_acc: 0.4073
Epoch 6/10
22967/22967 [==============================] - 45s - loss: 1.3918 - acc: 0.4771 - val_loss: 1.6917 - val_acc: 0.4227
Epoch 7/10
22967/22967 [==============================] - 51s - loss: 1.4012 - acc: 0.4693 - val_loss: 1.5832 - val_acc: 0.4133
Epoch 8/10
22967/22967 [==============================]

In [52]:
import keras
from keras.applications.vgg19 import VGG19
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model2 = Sequential()
model2.add(Conv2D(filters=8, kernel_size=2,padding='same', activation='relu', input_shape=img_input[train_indices].shape[1:]))
model2.add(MaxPooling2D(pool_size=2))
model2.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2))
model2.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=2))
model2.add(Dense(256, activation='relu'))
model2.add(GlobalAveragePooling2D())
model2.add(Dense(7, activation='softmax'))
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 48, 48, 8)         104       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 24, 24, 8)         0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 24, 24, 16)        528       
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 12, 12, 32)        2080      
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
dense_19 (Dense)             (None, 6, 6, 256)         8448      
__________

In [53]:
from keras.callbacks import ModelCheckpoint

model2.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='C:/Users/Alvin/.keras/models/weights.best.scratch2.face-{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

model2.fit(img_input[train_indices], train_labels,
          epochs=10,
          batch_size=20,
          validation_data=(img_input[valid_indices], valid_labels), callbacks = [checkpointer])
model2.save_weights('bottleneck_fc_model.h5')

Train on 22967 samples, validate on 5742 samples
Epoch 1/10
22967/22967 [==============================] - 29s - loss: 1.8424 - acc: 0.2576 - val_loss: 1.7829 - val_acc: 0.2816
Epoch 2/10
22967/22967 [==============================] - 29s - loss: 1.6992 - acc: 0.3095 - val_loss: 1.6771 - val_acc: 0.3333
Epoch 3/10
22967/22967 [==============================] - 26s - loss: 1.6450 - acc: 0.3427 - val_loss: 1.6624 - val_acc: 0.3380
Epoch 4/10
22967/22967 [==============================] - 27s - loss: 1.5984 - acc: 0.3686 - val_loss: 1.6021 - val_acc: 0.3757
Epoch 5/10
22967/22967 [==============================] - 22s - loss: 1.5613 - acc: 0.3897 - val_loss: 1.6038 - val_acc: 0.3570
Epoch 6/10
22967/22967 [==============================] - 22s - loss: 1.5307 - acc: 0.4006 - val_loss: 1.5216 - val_acc: 0.4143
Epoch 7/10
22967/22967 [==============================] - 26s - loss: 1.5025 - acc: 0.4204 - val_loss: 1.5135 - val_acc: 0.4140
Epoch 8/10
22967/22967 [==============================]

In [23]:
import keras
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model3 = Sequential()
model3.add(Conv2D(filters=16, kernel_size=2,padding='same', activation='relu', input_shape=img_input[train_indices].shape[1:]))
model3.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model3.add(MaxPooling2D(pool_size=2))
model3.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model3.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model3.add(MaxPooling2D(pool_size=2))
model3.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model3.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model3.add(MaxPooling2D(pool_size=2))
model3.add(Dense(512, activation='relu'))
model3.add(GlobalAveragePooling2D())
model3.add(Dense(7, activation='softmax'))
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 48, 48, 16)        208       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 48, 48, 16)        1040      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 24, 24, 32)        2080      
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 24, 24, 32)        4128      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 12, 12, 64)        8256      
__________

In [25]:
from keras.callbacks import ModelCheckpoint

model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='models/weights.best.scratch2.face-{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

model3.fit(img_input[train_indices], train_labels,
          epochs=10,
          batch_size=20,
          validation_data=(img_input[valid_indices], valid_labels), callbacks = [checkpointer])
model3.save_weights('bottleneck_fc_model.h5')

Train on 22967 samples, validate on 5742 samples
Epoch 1/10
22967/22967 [==============================] - 74s - loss: 1.0843 - acc: 0.5887 - val_loss: 1.2960 - val_acc: 0.5200
Epoch 2/10
22967/22967 [==============================] - 74s - loss: 1.0540 - acc: 0.5993 - val_loss: 1.3098 - val_acc: 0.5232
Epoch 3/10
22967/22967 [==============================] - 74s - loss: 1.0254 - acc: 0.6146 - val_loss: 1.3112 - val_acc: 0.5265
Epoch 4/10
22967/22967 [==============================] - 74s - loss: 0.9917 - acc: 0.6239 - val_loss: 1.3251 - val_acc: 0.5155
Epoch 5/10
22967/22967 [==============================] - 74s - loss: 0.9526 - acc: 0.6399 - val_loss: 1.3016 - val_acc: 0.5362
Epoch 6/10
22967/22967 [==============================] - 74s - loss: 0.9309 - acc: 0.6430 - val_loss: 1.4625 - val_acc: 0.5150
Epoch 7/10
22967/22967 [==============================] - 74s - loss: 0.8895 - acc: 0.6628 - val_loss: 1.3735 - val_acc: 0.5263
Epoch 8/10
22967/22967 [==============================]

In [34]:
import keras
from keras.applications.vgg19 import VGG19
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model4 = Sequential()
model4.add(Conv2D(filters=16, kernel_size=2,padding='same', activation='relu', input_shape=img_input[train_indices].shape[1:]))
model4.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model4.add(MaxPooling2D(pool_size=2))
model4.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model4.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model4.add(MaxPooling2D(pool_size=2))
model4.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model4.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model4.add(MaxPooling2D(pool_size=2))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(128, activation='relu'))
model4.add(Dropout(0.5))
model4.add(GlobalAveragePooling2D())
model4.add(Dense(7, activation='softmax'))
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_78 (Conv2D)           (None, 48, 48, 16)        208       
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 48, 48, 16)        1040      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 24, 24, 32)        2080      
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 24, 24, 32)        4128      
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 12, 12, 64)        8256      
__________

In [35]:
from keras.callbacks import ModelCheckpoint

model4.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='models/weights.best.scratch2.face-{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

model4.fit(img_input[train_indices], train_labels,
          epochs=10,
          batch_size=20,
          validation_data=(img_input[valid_indices], valid_labels), callbacks = [checkpointer])
model4.save_weights('bottleneck_fc_model.h5')

Train on 22967 samples, validate on 5742 samples
Epoch 1/10
22967/22967 [==============================] - 74s - loss: 1.7929 - acc: 0.2561 - val_loss: 1.7447 - val_acc: 0.2790
Epoch 2/10
22967/22967 [==============================] - 73s - loss: 1.6618 - acc: 0.3277 - val_loss: 1.5602 - val_acc: 0.3690
Epoch 3/10
22967/22967 [==============================] - 73s - loss: 1.5055 - acc: 0.4094 - val_loss: 1.4334 - val_acc: 0.4396
Epoch 4/10
22967/22967 [==============================] - 73s - loss: 1.4143 - acc: 0.4553 - val_loss: 1.3778 - val_acc: 0.4671
Epoch 5/10
22967/22967 [==============================] - 73s - loss: 1.3410 - acc: 0.4873 - val_loss: 1.3542 - val_acc: 0.4739
Epoch 6/10
22967/22967 [==============================] - 73s - loss: 1.2795 - acc: 0.5126 - val_loss: 1.3260 - val_acc: 0.4890
Epoch 7/10
22967/22967 [==============================] - 73s - loss: 1.2359 - acc: 0.5292 - val_loss: 1.3032 - val_acc: 0.5031
Epoch 8/10
22967/22967 [==============================]

In [48]:
import keras
from keras.applications.vgg19 import VGG19
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

model5 = Sequential()
model5.add(Conv2D(filters=16, kernel_size=2,padding='same', activation='relu', input_shape=img_input[train_indices].shape[1:]))
model5.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu'))
model5.add(MaxPooling2D(pool_size=2))
model5.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model5.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model5.add(MaxPooling2D(pool_size=2))
model5.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model5.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model5.add(MaxPooling2D(pool_size=2))
model5.add(Dense(128, activation='relu'))
model5.add(Dense(256, activation='relu'))
model5.add(GlobalAveragePooling2D())
#model5.add(Dropout(0.5))
model5.add(Dense(7, activation='softmax'))
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_121 (Conv2D)          (None, 48, 48, 16)        208       
_________________________________________________________________
conv2d_122 (Conv2D)          (None, 48, 48, 16)        1040      
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 24, 24, 32)        2080      
_________________________________________________________________
conv2d_124 (Conv2D)          (None, 24, 24, 32)        4128      
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_125 (Conv2D)          (None, 12, 12, 64)        8256      
__________

In [49]:
from keras.callbacks import ModelCheckpoint

model5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='models/weights.best.scratch2.face-{epoch:02d}-{val_loss:.2f}.hdf5', 
                               verbose=1, save_best_only=True)

model5.fit(img_input[train_indices], train_labels,
          epochs=10,
          batch_size=20,
          validation_data=(img_input[valid_indices], valid_labels), callbacks = [checkpointer])
model5.save_weights('bottleneck_fc_model.h5')

Train on 22967 samples, validate on 5742 samples
Epoch 1/10
22967/22967 [==============================] - 75s - loss: 1.7848 - acc: 0.2619 - val_loss: 1.7501 - val_acc: 0.2825
Epoch 2/10
22967/22967 [==============================] - 74s - loss: 1.6262 - acc: 0.3465 - val_loss: 1.5250 - val_acc: 0.3793
Epoch 3/10
22967/22967 [==============================] - 74s - loss: 1.4592 - acc: 0.4279 - val_loss: 1.4986 - val_acc: 0.4258
Epoch 4/10
22967/22967 [==============================] - 74s - loss: 1.3445 - acc: 0.4838 - val_loss: 1.3506 - val_acc: 0.4794
Epoch 5/10
22967/22967 [==============================] - 74s - loss: 1.2684 - acc: 0.5146 - val_loss: 1.3007 - val_acc: 0.4955
Epoch 6/10
22967/22967 [==============================] - 74s - loss: 1.2092 - acc: 0.5396 - val_loss: 1.2737 - val_acc: 0.5164
Epoch 7/10
22967/22967 [==============================] - 74s - loss: 1.1637 - acc: 0.5560 - val_loss: 1.2786 - val_acc: 0.5216
Epoch 8/10
22967/22967 [==============================]